In [1]:
from rank_bm25 import BM25Okapi
from tqdm import tqdm
import utils
from transformers import AutoTokenizer
model_name = "/home/zhoushiqi/workplace/model/deepseek-coder-6.7b-instruct"  # 你可以换成任何Hugging Face模型名称

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

datasets = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/megadiff-single-function/process.jsonl")
corpus = [data['diff_context'] for data in datasets]
tokenize_corpus = [data['tokenize_diff_context'] for data in datasets]


bm25 = BM25Okapi(tokenize_corpus)

buggy = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/df4_process_data/one_function/1.2.jsonl")
answer = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/baseline_1.2_N100_T1_process.jsonl")[0]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
answer[b1['project'] + '_' + str(b1['bug_id'])][50]

'\n    public void atan2(final double[] y, final int yOffset,\n                      final double[] x, final int xOffset,\n                      final double[] result, final int resultOffset) {\n\n        // compute r = sqrt(x^2+y^2)\n        double[] tmp1 = new double[getSize()];\n        multiply(x, xOffset, x, xOffset, tmp1, 0);      // x^2\n        double[] tmp2 = new double[getSize()];\n        multiply(y, yOffset, y, yOffset, tmp2, 0);      // y^2\n        add(tmp1, 0, tmp2, 0, tmp2, 0);                 // x^2 + y^2\n        rootN(tmp2, 0, 2, tmp1, 0);                     // r = sqrt(x^2 + y^2)\n\n        if (x[xOffset] >= 0) {\n\n            // compute atan2(y, x) = 2 atan(y / (r + x))\n            add(tmp1, 0, x, xOffset, tmp2, 0);          // r + x\n            divide(y, yOffset, tmp2, 0, tmp1, 0);       // y /(r + x)\n            atan(tmp1, 0, tmp2, 0);                     // atan(y / (r + x))\n            for (int i = 0; i < tmp2.length; ++i) {\n                result[result

In [20]:
from collections import defaultdict
import tqdm
bug_id = 0
all_retrieval = defaultdict(int)
for i in tqdm.tqdm(range(100)):
    fix_id = i
    b1 = buggy[bug_id]
    a1 = answer[b1['project'] + '_' + str(b1['bug_id'])][fix_id]
    data = {'buggy_function':b1['erro_repairs'][0]['src_code'][0], 'fixed_function':a1.strip('\n')}
    query = utils._process(data, 3)
    tokenized_query = tokenizer.tokenize(query)
    retrive = bm25.get_top_n(tokenized_query, corpus, n=10)
    for r in retrive:
        all_retrieval[r] += 1

100%|██████████| 100/100 [28:53<00:00, 17.34s/it]


In [22]:
order = sorted(all_retrieval, key=lambda x:all_retrieval[x], reverse=True)

In [3]:

from tqdm import tqdm
import utils
from transformers import AutoTokenizer
model_name = "/home/zhoushiqi/workplace/model/deepseek-coder-6.7b-instruct"  # 你可以换成任何Hugging Face模型名称

# 加载tokenizerfrom gensim_bm25 import bm25
tokenizer = AutoTokenizer.from_pretrained(model_name)

datasets = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/megadiff-single-function/process.jsonl")
documents = [data['buggy_function'] for data in datasets]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import tqdm
new_datasets = []
index = 0
for data in tqdm(datasets):
    new_data = {}
    b = data["buggy_function"]
    b = b.strip()
    b = b.replace('\t', '    ')
    tokenized_buggy_code = tokenizer.tokenize(b)
    if len(tokenized_buggy_code) > 10000:
        continue
    f = data["fixed_function"]
    f = f.strip()
    f = f.replace('\t', '    ')
    tokenized_fixed_code = tokenizer.tokenize(f)
    new_data['buggy_function'] = b
    new_data['fixed_function'] = f
    new_data['diff_context'] = data["diff_context"]
    new_data['tokenized_diff_context'] = data["tokenize_diff_context"]
    new_data['tokenized_buggy_function'] = tokenized_buggy_code
    new_data['tokenized_fixed_function'] = tokenized_fixed_code
    new_datasets.append(new_data)

NameError: name 'datasets' is not defined

In [ ]:
utils.write_jsonl("/home/zhoushiqi/workplace/apr/data/megadiff-single-function/process.jsonl", new_datasets)

In [16]:
buggy = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/df4_process_data/one_function/1.2.jsonl")
answer = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_OnlyBuggycode_N100_T1_process.jsonl")[0]

In [17]:
diff_datas = []
for bug in tqdm.tqdm(buggy):
    new_data = {}
    bug_id = f"{bug['project']}_{bug['bug_id']}"
    if bug_id in plausible_ids or bug_id not in answer:
        continue
    buggy_code = bug['erro_repairs'][0]['src_code'][0].strip()
    new_data['project'] = bug['project']
    new_data['bug_id'] = bug['bug_id']
    new_data['buggy_code'] = buggy_code
    new_data['fixed'] = [a.strip() for a in answer[bug_id]]
    new_data['diffs'] = [utils._process({'buggy_function':buggy_code, 'fixed_function':a}, 3) for a in new_data['fixed']]
    diff_datas.append(new_data)

100%|██████████| 255/255 [00:13<00:00, 18.41it/s]


In [1]:
import json
x = json.load(open("/home/zhoushiqi/workplace/minicpm/MiniCPM/finetune/data/AdvertiseGen/all.json"))

In [6]:
import utils
# y1 = utils.read_jsonl("/home/guochuanzhe/share/math_sft/MathInstruct/MathInstruct.jsonl")
# y2 = utils.read_jsonl("/home/guochuanzhe/share/math_sft/MetaMathQA/MetaMathQA.jsonl")
y3 = utils.read_jsonl("/home/guochuanzhe/share/math_sft/orca-math-word-problems-200k/orca-math-word-problems-200k.jsonl")
# y4 = utils.read_jsonl("/home/guochuanzhe/share/math_sft/reasoningdata_200k-sharegpt/reasoningdata_200k-sharegpt.jsonl")

In [10]:
import random
random.seed(42)
add = random.sample(y3, k=len(x)*2//8)

In [12]:
x += add

In [15]:
json.dump(x, open("/home/zhoushiqi/workplace/minicpm/MiniCPM/finetune/data/AdvertiseGen/all_add_math_8&2.json", 'w'))

In [19]:
json.dump(x, open("/home/zhoushiqi/workplace/minicpm/MiniCPM/finetune/data/AdvertiseGen/all_add_math.json", 'w'))

In [2]:
import utils
len(utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_random_N200_T1_process.jsonl")[0])

223

In [34]:
import numpy as np
from gensim.similarities.docsim import MatrixSimilarity
# 创建一些简单的文档向量
documents_vectors = [[(0,0.121),(1,0.2)],[(0,0.2),(1,0.2)]]

# 创建相似度索引
index = MatrixSimilarity(documents_vectors, num_features=2)

# 创建一个简单的查询向量
query_vector = [(0,0.1),(1,0.1)]

# 执行查询
sims = index[query_vector]

# 输出查询结果
print(sims)
index.save("temp")


[0.9710257  0.99999994]


In [39]:
import utils
for i,x in enumerate(utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/megadiff-single-function/process_filtered2048.jsonl")):
    assert i == x['index']

In [26]:
plausible = set()

In [28]:
from utils import read_jsonl
x = read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_bm25_k1_add_report_N200_T1_result.jsonl")
t = 0
for xx in x:
    if "other test Failing tests: 0\n" in xx['result'][:100]:
        plausible.add(f"{xx['project']}_{xx['bug_id']}")
        t += 1
t

98

In [29]:
len(plausible)

113

In [18]:
def _split_bug_fix(diff):
    bug = []
    fix = []
    for l in diff.split('\n'):
        if l.startswith('+'):
            fix.append(l[1:])
        elif l.startswith('-'):
            bug.append(l[1:])
        else:
            fix.append(l[1:])
            bug.append(l[1:])
    return '\n'.join(bug), '\n'.join(fix)

In [30]:
import json
with open("/home/zhoushiqi/workplace/apr/data/megadiff-single-function/process_filtered2048_add_fl_bfp.jsonl") as f:
    i = 0
    for l in f:
        if i > 0:
            break
        l = json.loads(l)
        print(l)
        print('==================')
        i +=1


{'buggy_function': 'private void ping() throws IOException, InterruptedException {\n        Future<?> f = channel.callAsync(new Ping());\n        long start = System.currentTimeMillis();\n        long end = start +timeout;\n\n        long remaining;\n        do {\n            remaining = end-System.currentTimeMillis();\n            try {\n                f.get(Math.max(0,remaining),MILLISECONDS);\n                return;\n            } catch (ExecutionException e) {\n                if (e.getCause() instanceof RequestAbortedException)\n                    return; // connection has shut down orderly.\n                onDead(e);\n                return;\n            } catch (TimeoutException e) {\n                // get method waits "at most the amount specified in the timeout",\n                // so let\'s make sure that it really waited enough\n            }\n        } while(remaining>0);\n\n        onDead(new TimeoutException("Ping started on "+start+" hasn\'t completed at "+System.c

In [1]:
import utils
import json

In [2]:
result_ours = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_diff_vec_add_report_k1_N100_T1_result.jsonl")
result_cedar = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_buggy_vector_k1_add_report_sroberta_N200_T1_result.jsonl")
result_random = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_random_add_report_N200_T1_result.jsonl")

In [3]:
result_ours = {f"{x['project']}_{x['bug_id']}":x for x in result_ours}
result_cedar = {f"{x['project']}_{x['bug_id']}":x for x in result_cedar}
result_random = {f"{x['project']}_{x['bug_id']}":x for x in result_random}

In [4]:
candidate = {}
for k in result_ours:
    if "other test Failing tests: 0\n" in result_ours[k]['result'] and\
        "other test Failing tests: 0\n" not in result_cedar[k]['result'] and\
        "other test Failing tests: 0\n" not in result_random[k]['result']:
        candidate[k] = len(result_ours[k]['result'])

In [6]:
order = sorted(list(candidate.keys()), key=lambda x:candidate[x])

In [20]:
prompt_ours = {f"{x['project']}_{x['bug_id']}":x['prompt'] for x in utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_diff_vec_add_report_k1_N100_T1.jsonl")}
prompt_cedar = {f"{x['project']}_{x['bug_id']}":x['prompt'] for x in utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_buggy_vector_k1_add_report_sroberta_N200_T1.jsonl")}
prompt_random = {f"{x['project']}_{x['bug_id']}":x['prompt'] for x in utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_random_add_report_N200_T1.jsonl")}

In [25]:
infos = {f"{x['project']}_{x['bug_id']}":x for x in utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/df4_process_data/one_function/1.2.jsonl")}

In [26]:
infos['Math_10']

{'version': 'defects4j-1.2.0',
 'project': 'Math',
 'bug_id': 10,
 'Test_file_path': '/home/zhoushiqi/workplace/apr/df4/all_project/defects4j-1.2.0/Math/10/src/test/java/org/apache/commons/math3/analysis/differentiation/DerivativeStructureTest.java',
 'local_dir_path': '/home/zhoushiqi/workplace/apr/data/df4_process_data/defects4j-1.2.0/Math/10/',
 'trigger_line': ['                                          new DerivativeStructure(2, 2, 1, +0.0));',
  '        Assert.assertEquals(0, pp.getValue(), 1.0e-15);'],
 'test_func': '    @Test\n    public void testAtan2SpecialCases() {\n\n        DerivativeStructure pp =\n                DerivativeStructure.atan2(new DerivativeStructure(2, 2, 1, +0.0),\n                                          new DerivativeStructure(2, 2, 1, +0.0));\n        Assert.assertEquals(0, pp.getValue(), 1.0e-15);//The program encountered an error here\n        Assert.assertEquals(+1, FastMath.copySign(1, pp.getValue()), 1.0e-15);\n\n        DerivativeStructure pn =\n

In [28]:
import difflib

In [31]:
for k in order:
    print(k)
    print("================ground truth==============")
    bfp = infos[k]
    print('---------------diff----------')
    print('\n'.join(list(difflib.unified_diff(bfp['erro_repairs'][0]['src_code'][0].splitlines(), bfp['erro_repairs'][0]['fixs'][0].splitlines(), n=3))))
    print("================ours retrieve================")
    prompt = prompt_ours[k][prompt_ours[k].index("### Example 1"):prompt_ours[k].index("### Example 2")]
    bug = prompt[prompt.index("### Buggy code:"):prompt.index("### Refined code:")].strip().splitlines()[1:]
    fix = prompt[prompt.index("### Refined code:"):].strip().splitlines()[1:]
    print('\n'.join(list(difflib.unified_diff(bug, fix, n=3))))
    print("================cedar retrieve================")
    prompt = prompt_cedar[k][prompt_cedar[k].index("### Example 1"):prompt_cedar[k].index("### Example 2")]
    bug = prompt[prompt.index("### Buggy code:"):prompt.index("### Refined code:")].strip().splitlines()[1:]
    fix = prompt[prompt.index("### Refined code:"):].strip().splitlines()[1:]
    print('\n'.join(list(difflib.unified_diff(bug, fix, n=3))))
    print("================random retrieve================")
    prompt = prompt_random[k][prompt_random[k].index("### Example 1"):prompt_random[k].index("### Example 2")]
    bug = prompt[prompt.index("### Buggy code:"):prompt.index("### Refined code:")].strip().splitlines()[1:]
    fix = prompt[prompt.index("### Refined code:"):].strip().splitlines()[1:]
    print('\n'.join(list(difflib.unified_diff(bug, fix, n=3))))

Lang_24
================ground truth==============
---------------diff----------
--- 

+++ 

@@ -92,7 +92,7 @@

             if (chars[i] == 'l'
                 || chars[i] == 'L') {
                 // not allowing L with an exponent or decimal point
-                return foundDigit && !hasExp;
+                return foundDigit && !hasExp && !hasDecPoint;
             }
             // last character is illegal
             return false;
================ours retrieve================
--- 

+++ 

@@ -10,7 +10,7 @@

     numDigits = 0;
     for (int i = 0; i < length; i++) {
         char c = s.charAt(i);
-        if(c >= '0' && c <= '9') {
+        if (c >= '0' && c <= '9') {
             /* Can't have more than 3 digits per octet. */
             if (numDigits == 3)
                 return null;
================cedar retrieve================
--- 

+++ 

@@ -1,5 +1,5 @@

 ```java
-public boolean isNullOrEmpty(String str){
+public static boolean isNullOrEmpty(String str){
         re

In [44]:
y = infos['Closure_104']['erro_repairs'][0]['src_code'][0].splitlines()

In [37]:
ours_process = json.load(open("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_diff_vec_add_report_k1_N100_T1_process.jsonl"))

In [52]:
for x in ours_process['Closure_104'][12:14]:
    print('---------------diff----------')
    print('\n'.join(list(difflib.unified_diff(y, x.strip().splitlines()))))

---------------diff----------
--- 

+++ 

@@ -1,4 +1,4 @@

-  JSType meet(JSType that) {
+JSType meet(JSType that) {
     UnionTypeBuilder builder = new UnionTypeBuilder(registry);
     for (JSType alternate : alternates) {
       if (alternate.isSubtype(that)) {
@@ -10,6 +10,12 @@

       for (JSType otherAlternate : ((UnionType) that).alternates) {
         if (otherAlternate.isSubtype(this)) {
           builder.addAlternate(otherAlternate);
+        }
+      }
+    } else if (this instanceof UnionType) {
+      for (JSType myAlternate : ((UnionType) this).alternates) {
+        if (myAlternate.isSubtype(that)) {
+          builder.addAlternate(myAlternate);
         }
       }
     } else if (that.isSubtype(this)) {
---------------diff----------
--- 

+++ 

@@ -1,4 +1,7 @@

-  JSType meet(JSType that) {
+JSType meet(JSType that) {
+    if (this == that || that.isSubtype(this)) {
+      return that;
+    }
     UnionTypeBuilder builder = new UnionTypeBuilder(registry);
     for (JST

In [53]:
y[0]

'  JSType meet(JSType that) {'

In [58]:
print('\n'.join(list(difflib.unified_diff(ours_process['Closure_104'][12].strip().splitlines(), y))))

--- 

+++ 

@@ -1,4 +1,4 @@

-JSType meet(JSType that) {
+  JSType meet(JSType that) {
     UnionTypeBuilder builder = new UnionTypeBuilder(registry);
     for (JSType alternate : alternates) {
       if (alternate.isSubtype(that)) {
@@ -10,12 +10,6 @@

       for (JSType otherAlternate : ((UnionType) that).alternates) {
         if (otherAlternate.isSubtype(this)) {
           builder.addAlternate(otherAlternate);
-        }
-      }
-    } else if (this instanceof UnionType) {
-      for (JSType myAlternate : ((UnionType) this).alternates) {
-        if (myAlternate.isSubtype(that)) {
-          builder.addAlternate(myAlternate);
         }
       }
     } else if (that.isSubtype(this)) {


In [60]:
ours_process['Closure_104'][12].strip().splitlines()[12:22]

['        }',
 '      }',
 '    } else if (this instanceof UnionType) {',
 '      for (JSType myAlternate : ((UnionType) this).alternates) {',
 '        if (myAlternate.isSubtype(that)) {',
 '          builder.addAlternate(myAlternate);',
 '        }',
 '      }',
 '    } else if (that.isSubtype(this)) {',
 '      builder.addAlternate(that);']

In [61]:
y

['  JSType meet(JSType that) {',
 '    UnionTypeBuilder builder = new UnionTypeBuilder(registry);',
 '    for (JSType alternate : alternates) {',
 '      if (alternate.isSubtype(that)) {',
 '        builder.addAlternate(alternate);',
 '      }',
 '    }',
 '',
 '    if (that instanceof UnionType) {',
 '      for (JSType otherAlternate : ((UnionType) that).alternates) {',
 '        if (otherAlternate.isSubtype(this)) {',
 '          builder.addAlternate(otherAlternate);',
 '        }',
 '      }',
 '    } else if (that.isSubtype(this)) {',
 '      builder.addAlternate(that);',
 '    }',
 '    JSType result = builder.build();',
 '    if (result != null) {',
 '      return result;',
 '    } else if (this.isObject() && that.isObject()) {',
 '      return getNativeType(JSTypeNative.NO_OBJECT_TYPE);',
 '    } else {',
 '      return getNativeType(JSTypeNative.NO_TYPE);',
 '    }',
 '  }']